# POVa - Facial recognition from RGB-Depth images (identification)

Our goal is to get some face recognition working using RGB-D data (e.g. Kinect).

- Detect faces in images using an existing detector. Good choices are OpenCV, Dlib or MTCNN https://github.com/DCGM/mtcnn.
- Align the face based on detected facial features (map to an average face).
- Optional: (Try align 3D face pose using the depth data)
- Train a neural network to identify faces.

In [1]:
import os

dataset_name = "atulanandjha/lfwpeople"
DATA_PATH = "../data/" + dataset_name

Run only if you don't have the dataset already in your project.
________

In [ ]:
import kagglehub

path = kagglehub.dataset_download(dataset_name)
print("\n"+path)

In [ ]:
%mkdir -p $DATA_PATH
%mv $path/* $DATA_PATH

In [ ]:
!tar -xvzf $DATA_PATH/*.tgz -C $DATA_PATH

In [5]:
import random
import shutil

if not os.path.isdir("../data/dataset/"):
    os.mkdir("../data/dataset/")
    os.mkdir("../data/dataset/training")
    os.mkdir("../data/dataset/test")

for base_path, dirs, _ in os.walk("../data/" + dataset_name + "/lfw_funneled"):
    for dir in dirs:
        if len(os.listdir(os.path.join(base_path, dir))) >= 5:
            if not os.path.isdir(os.path.join("../data/dataset/training", dir)):
                os.mkdir(os.path.join("../data/dataset/training", dir))
    
            file_list = os.listdir(os.path.join(base_path, dir))
            random.shuffle(file_list)
    
            split = int(0.8 * len(file_list))
    
            for file_name in file_list[0:split]:
                full_img_path = os.path.join(base_path, dir, file_name)
                if os.path.isfile(full_img_path):
                    shutil.copy(full_img_path, os.path.join("../data/dataset/training", dir))
        
            for file_name in file_list[split:]:
                full_img_path = os.path.join(base_path, dir, file_name)
                if os.path.isfile(full_img_path):
                    shutil.copy(full_img_path, "../data/dataset/test")

_____________________

## LFW - People (Face Recognition) Dataset

https://vis-www.cs.umass.edu/lfw/

In [ ]:
%pip install mtcnn tensorflow keras

In [ ]:
from mtcnn import MTCNN
import cv2

detector = MTCNN()
image = cv2.imread("../data/dataset/training/Abdullah_Gul/Abdullah_Gul_0001.jpg")
faces = detector.detect_faces(image)
for face in faces:
    box = face['box']
    cv2.rectangle(image, (box[0], box[1]), (box[0]+box[2], box[1]+box[3]), (255, 0, 0), 2)
cv2.imshow("Detected Faces", image)
cv2.waitKey(0) # Escape key
cv2.destroyAllWindows()

Display the number of different classes:

In [ ]:
n_labels = 0
for root, dirs, files in os.walk(DATA_PATH + "/lfw_funneled/"):
    n_labels += len(dirs)
    break

print("Number of classes: ", n_labels)

________________________

## Face Detection

In [43]:
from sklearn.datasets import fetch_lfw_people
from sklearn.model_selection import train_test_split

lfw_people = fetch_lfw_people(min_faces_per_person=40, resize=1.0)
X = lfw_people.images
y = lfw_people.target
target_names = lfw_people.target_names

n_samples, width, height = lfw_people.images.shape
n_features = X.shape[1]
n_classes = target_names.shape[0]

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
print("Total dataset size:")
print("n_samples: %d" % n_samples)
print("n_features: %d" % n_features)
print("n_classes: %d" % n_classes)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(1)
figs, axes = plt.subplots(4, 6)
for i in range(4):
    for j in range(6): 
        axes[i, j].imshow(X[i*6+j,:,:])
        axes[i, j].set_xticks([])
        axes[i, j].set_yticks([])
plt.show()

In [46]:
from tensorflow.keras.utils import to_categorical

# normalization
x_train = x_train.astype('float32') / 255.0
x_test  = x_test.astype( 'float32') / 255.0

# convert integer labesls to categorical vectors
y_train = to_categorical(y_train, n_labels)
y_test  = to_categorical(y_test, n_labels)

## Facial Feature Alignment

## CNN model

In [44]:
batch_size = 64
epochs     = 100
l_rate      = 2e-4

In [45]:
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras import optimizers
import tensorflow as tf

def baseline(W=32, H=32, nclass=10, nchannel=3,lr=1e-4):
    in1 = layers.Input(shape=(W, H, nchannel))
    x = layers.Conv2D(32, (3, 3), strides=(1, 1),
                      padding='valid',
                      activation='relu')(in1)    
    x = layers.MaxPool2D((2, 2))(x)
    x = layers.BatchNormalization()(x)    
    x = layers.Conv2D(32, (3, 3), strides=(1, 1),
                      padding='valid',
                      activation='relu')(x)    
    x = layers.BatchNormalization()(x)
    x = layers.MaxPool2D((2, 2))(x)    
    x = layers.Conv2D(64, (3, 3), strides=(1, 1),
                      padding='valid',
                      activation='relu')(x)
    x = layers.MaxPool2D((2, 2))(x)
    x = layers.Flatten()(x)
    x = layers.Dropout(0.4)(x)
    # x = layers.Dense(128, activation='relu')(x)
    output = layers.Dense(nclass, activation='softmax')(x)
    model = Model(inputs=in1, outputs=output)
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.Adam(learning_rate=lr),
                  metrics=['acc'])
    return model

In [46]:
checkpoint_filepath = './checkpoints/checkpoint.weights.h5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_acc',
    mode='max',
    save_best_only=True
)

In [ ]:
model_base = baseline(W=width, H=height, nclass=n_labels ,nchannel=1, lr=l_rate)
model_base.summary()

In [ ]:
history = model_base.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, callbacks=[model_checkpoint_callback], validation_split=0.2, verbose=False)

model_base.load_weights(checkpoint_filepath)   
test_loss, test_acc = model_base.evaluate(x_test, y_test)    
print('test acc for model_base: ', test_acc)

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss'] 

plt.figure(2)
epochs = range(len(acc))
plt.plot(epochs, acc, 'r', label='Training acc - baseline')
plt.plot(epochs, val_acc, 'm:', label='Validation acc - baseline')

plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.grid()
plt.legend()
plt.show()

In [ ]:
plt.figure(3)
plt.plot(epochs, loss, 'r',  label='Training loss - baseline')
plt.plot(epochs, val_loss, 'm:', label='Validation loss - baseline')
plt.title( 'Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid()
plt.show()